# START YOUR CODE HERE IN THIS NOTEBOOK, SIR/MS. :)

In [ ]:
import os

modelList = [] 

for x in os.listdir("../model"): 
    modelList += [x]
print(modelList)

In [ ]:
from ipywidgets import interact, Dropdown 
 
models = Dropdown(options = modelList) 
 
@interact(model = models) 
def print_city(model): 
    print(model)